In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="7UTVIkgLjhn1CHSCwRRD")
project = rf.workspace("kaushik-ubnqt").project("groceries-object-detection")
version = project.version(1)
dataset = version.download("folder")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 93.4 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to groceries-object-detection-1 in folder:: 100%|██████████| 11921/11921 [00:01<00:00, 10234.59it/s]


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import tensorflow as tf
import json
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import load_img, img_to_array


In [13]:
DATASET_PATH = "/content/groceries-object-detection-1"
IMG_SIZE = (224, 224)
BATCH_SIZE = 16
EPOCHS = 20

MODEL_DRIVE_PATH = "/content/drive/MyDrive/grocery_classifier_mobilenetv2.h5"
CLASS_PATH = "/content/drive/MyDrive/class_names.json"


In [5]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

valid_datagen = ImageDataGenerator(rescale=1./255)


In [6]:
train_generator = train_datagen.flow_from_directory(
    f"{DATASET_PATH}/train",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_generator = valid_datagen.flow_from_directory(
    f"{DATASET_PATH}/valid",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

test_generator = valid_datagen.flow_from_directory(
    f"{DATASET_PATH}/test",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

NUM_CLASSES = train_generator.num_classes
print("Number of classes:", NUM_CLASSES)


Found 10362 images belonging to 25 classes.
Found 984 images belonging to 25 classes.
Found 495 images belonging to 25 classes.
Number of classes: 25


In [7]:
class_names = list(train_generator.class_indices.keys())

with open(CLASS_PATH, "w") as f:
    json.dump(class_names, f)

print("Class names saved to Drive ✅")


Class names saved to Drive ✅


In [8]:
base_model = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)

base_model.trainable = False


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [9]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.3)(x)

outputs = Dense(NUM_CLASSES, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=outputs)


In [10]:
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,425,177 (9.25 MB)

 Trainable params: 167,193 (653.10 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [11]:
es = EarlyStopping(patience=3, restore_best_weights=True)
rlr = ReduceLROnPlateau(factor=0.2, patience=2)


In [14]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=[es, rlr]
)


Epoch 1/20
648/648 ━━━━━━━━━━━━━━━━━━━━ 150s 231ms/step - accuracy: 0.4434 - loss: 1.8582 - val_accuracy: 0.6179 - val_loss: 1.2332 - learning_rate: 0.0010
Epoch 2/20
648/648 ━━━━━━━━━━━━━━━━━━━━ 123s 191ms/step - accuracy: 0.5578 - loss: 1.4412 - val_accuracy: 0.6372 - val_loss: 1.1521 - learning_rate: 0.0010
Epoch 3/20
648/648 ━━━━━━━━━━━━━━━━━━━━ 123s 190ms/step - accuracy: 0.6106 - loss: 1.2624 - val_accuracy: 0.6636 - val_loss: 1.0637 - learning_rate: 0.0010
Epoch 4/20
648/648 ━━━━━━━━━━━━━━━━━━━━ 124s 191ms/step - accuracy: 0.6522 - loss: 1.1254 - val_accuracy: 0.6707 - val_loss: 1.0293 - learning_rate: 0.0010
Epoch 5/20
648/648 ━━━━━━━━━━━━━━━━━━━━ 122s 189ms/step - accuracy: 0.6662 - loss: 1.0747 - val_accuracy: 0.6738 - val_loss: 1.0527 - learning_rate: 0.0010
Epoch 6/20
648/648 ━━━━━━━━━━━━━━━━━━━━ 124s 191ms/step - accuracy: 0.6749 - loss: 1.0219 - val_accuracy: 0.6728 - val_loss: 1.0518 - learning_rate: 0.0010
Epoch 7/20
648/648 ━━━━━━━━━━━━━━━━━━━━ 124s 191ms/step - accura

In [15]:
 base_model.trainable = True
for layer in base_model.layers[:-50]:
    layer.trainable = False


In [16]:
model.compile(
    optimizer=Adam(1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [17]:
history_ft = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[es, rlr]
)


Epoch 1/20
648/648 ━━━━━━━━━━━━━━━━━━━━ 166s 223ms/step - accuracy: 0.4322 - loss: 2.1628 - val_accuracy: 0.7226 - val_loss: 0.9855 - learning_rate: 1.0000e-05
Epoch 2/20
648/648 ━━━━━━━━━━━━━━━━━━━━ 174s 197ms/step - accuracy: 0.5887 - loss: 1.3306 - val_accuracy: 0.7165 - val_loss: 0.9563 - learning_rate: 1.0000e-05
Epoch 3/20
648/648 ━━━━━━━━━━━━━━━━━━━━ 127s 196ms/step - accuracy: 0.6604 - loss: 1.0960 - val_accuracy: 0.7083 - val_loss: 0.9384 - learning_rate: 1.0000e-05
Epoch 4/20
648/648 ━━━━━━━━━━━━━━━━━━━━ 129s 198ms/step - accuracy: 0.6980 - loss: 0.9704 - val_accuracy: 0.7226 - val_loss: 0.9022 - learning_rate: 1.0000e-05
Epoch 5/20
648/648 ━━━━━━━━━━━━━━━━━━━━ 129s 199ms/step - accuracy: 0.7214 - loss: 0.8902 - val_accuracy: 0.7297 - val_loss: 0.8626 - learning_rate: 1.0000e-05
Epoch 6/20
648/648 ━━━━━━━━━━━━━━━━━━━━ 128s 198ms/step - accuracy: 0.7511 - loss: 0.8019 - val_accuracy: 0.7490 - val_loss: 0.8292 - learning_rate: 1.0000e-05
Epoch 7/20
648/648 ━━━━━━━━━━━━━━━━━━━━ 

In [18]:
test_loss, test_acc = model.evaluate(test_generator)
print("Test Accuracy:", test_acc)


31/31 ━━━━━━━━━━━━━━━━━━━━ 11s 366ms/step - accuracy: 0.8033 - loss: 0.6697
Test Accuracy: 0.8202019929885864


In [19]:
model.save(MODEL_DRIVE_PATH)
print("Model saved to Google Drive successfully ✅")


Model saved to Google Drive successfully ✅


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [20]:
from tensorflow.keras.models import load_model

In [21]:
model = load_model(MODEL_DRIVE_PATH)

with open(CLASS_PATH, "r") as f:
    class_names = json.load(f)

print("Model & classes loaded ✅")


Model & classes loaded ✅


In [22]:
def predict_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img) / 255.0
    img = np.expand_dims(img, axis=0)

    preds = model.predict(img)
    idx = np.argmax(preds[0])

    return class_names[idx], preds[0][idx]


In [24]:
pred_class, confidence = predict_image("/content/cack image.jpg")
print(f"Predicted: {pred_class}, Confidence: {confidence:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step
Predicted: cake, Confidence: 0.55
